[View in Colaboratory](https://colab.research.google.com/github/clee1994/DLclass/blob/master/HW02_Convolution.ipynb)

In [0]:
import numpy as np
from scipy import signal as signal
import matplotlib.pyplot as plt
import time
import tensorflow  as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
def tf2dConv(matrix_reshaped,kernel_reshaped):
  config = tf.ConfigProto(allow_soft_placement = True)
  config.gpu_options.allow_growth = True

  with tf.device('/gpu:0'):
    
    input_matrix =  tf.Variable(matrix_reshaped.astype(float))
    kernel_tf = tf.constant(kerne_reshaped.astype(float))

    convoluted_matrix = tf.nn.convolution(input=input_matrix,filter=kernel_tf,strides=[1,1],padding="VALID")
    init = tf.global_variables_initializer()

    with tf.Session(config=config) as sess:
      sess.run(init)
      
      time1 = time.time()
      output_matrix = sess.run(convoluted_matrix)
      time2 = time.time()
  return output_matrix[0],(time2-time1)



In [0]:
def tf2dConvCPU(matrix_reshaped,kernel_reshaped):
  config = tf.ConfigProto(allow_soft_placement = True)
  config.gpu_options.allow_growth = True

  with tf.device('/gpu:0'):
    
    input_matrix =  tf.Variable(matrix_reshaped.astype(float))
    kernel_tf = tf.constant(kerne_reshaped.astype(float))

    convoluted_matrix = tf.nn.convolution(input=input_matrix,filter=kernel_tf,strides=[1,1],padding="VALID")
    init = tf.global_variables_initializer()

    with tf.Session(config=config) as sess:
      sess.run(init)
      
      time1 = time.time()
      output_matrix = sess.run(convoluted_matrix)
      time2 = time.time()
  return output_matrix[0],(time2-time1)


In [0]:
def padWith(vector, pad_width, iaxis, kwargs):
  pad_value = kwargs.get('padder', 10)
  vector[:pad_width[0]] = pad_value
  vector[-pad_width[1]:] = pad_value
  return vector

In [0]:
def joshi2dpatch(patch,kernel):
  flippedKernel = np.fliplr(np.flipud(kernel))
  appliedKernel = np.multiply(patch,flippedKernel)
  outPixel = np.sum(np.sum(appliedKernel))
  
  return outPixel

In [0]:
def joshi2dConv(inMatrix,kernel):
  
  kSize = kernel.shape
  pSize = inMatrix.shape
  outputH = (pSize[0]-kSize[0])+1
  outputW = (pSize[1]-kSize[1])+1
  outputPatch = np.zeros([outputH,outputW])
  
  
  for outer in range(outputH):
    for inner in range(outputW):
      outputPatch[outer,inner] = joshi2dpatch(inMatrix[outer:(outer+kSize[0]),inner:(inner+kSize[1])],kernel) 
  
  return outputPatch

In [0]:
def runConvs(matrix, kernel):
  mshape = matrix.shape
  kshape = kernel.shape
  
  #Joshi
  time1 = time.time()
  testConvJoshi = joshi2dConv(matrix,kernel)
  time2 = time.time()
  print('Joshi Convolution per second: {:.3f}'.format(1/(time2-time1)))

  #Scipy
  time1 = time.time()
  testConvScipy = signal.convolve2d(matrix,kernel,mode='valid')
  time2 = time.time()
  print('Scipy Convolution per second: {:.3f}'.format(1/(time2-time1)))

  
  #tensorflow with GPU
  matrix_reshaped = matrix.reshape(1, mshape[0], mshape[1], 1)
  kernel_reshaped = kernel.reshape(kshape[0], kshape[1], 1, 1)
  testConvTf, tftime = tf2dConv(matrix_reshaped,kernel_reshaped)
  print('Tensorflow Convolution per second: {:.3f}'.format(1/tftime))
  
  

In [0]:
if __name__=='__main__':
  #test kernels
  test_kernel = np.eye(2)
  kernel3 = np.eye(3)
  kernel5 = np.eye(5)
  kernel7 = np.eye(7)
  
  
  #generate test data
  test_matrix = np.array([[1,2,3],[4,5,6],[7,8,9]])
  matrix28 = np.random.randn(28,28)*100
  matrix32 = np.random.randn(32,32)*100
  
  runConvs(matrix32,kernel3)
  
  
  



Joshi Convolution per second: 70.131
Scipy Convolution per second: 7810.622
Tensorflow Convolution per second: 173.462
